<a href="https://colab.research.google.com/github/bksat90/ImageSegmentation/blob/main/Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation of KerasCV.
!pip install -q --upgrade keras-cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
# import libraries
import random
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import keras
import keras_cv

from keras import layers
from keras_cv import bounding_box
from keras_cv import visualization
from pycocotools.coco import COCO

%matplotlib inline

Using TensorFlow backend


In [7]:
# Import and mount Google Drive.
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
# Unzip the dataset file and store it in a folder called images.
!unzip "/content/drive/MyDrive/RMiDS/RM_Segmentation_Assignment_dataset.zip" -d "/content/drive/MyDrive/RMiDS/coco2017/"

Archive:  /content/drive/MyDrive/RMiDS/RM_Segmentation_Assignment_dataset.zip
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001751.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001380.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001643.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001583.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001685.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001482.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001494.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001551.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001654.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001594.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/000000001411.jpg  
  inflating: /content/drive/MyDrive/RMiDS/coco2017/test-30/00000

In [10]:
# Define location of training data (images and labels).
train_data_path = "/content/drive/MyDrive/RMiDS/coco2017/train-300"
train_annotation_file = f"{train_data_path}/labels.json"

In [11]:
# Initialising COCO API for instance annotations
coco = COCO(train_annotation_file)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
